<a href="https://colab.research.google.com/github/finardi/WatSpeed_LLM_foundation/blob/main/Solution_Module_1_Using_BloomZ_7b_for_sentence_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 787.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Downloading Dataset

The SST-2 dataset, or the Stanford Sentiment Treebank, is popular for sentiment analysis tasks in Natural Language Processing (NLP). It consists of movie reviews from the Rotten Tomatoes website that are labeled with either a positive or negative sentiment. The dataset contains 10,662 sentence-level movie reviews, with approximately half of the reviews labeled as positive and the other half labeled as negative. The reviews are also relatively evenly distributed in length, with a median length of 18 tokens.

The SST-2 dataset has become a benchmark dataset for sentiment analysis in NLP, and many researchers use it to evaluate the performance of their models. The dataset's popularity is partly due to its high-quality labels and the task's relative simplicity, making it an accessible starting point for researchers and developers new to NLP.

In this example, we're using the **`datasets`** library to download and load the validation set of the dataset.

In [40]:
import torch

import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('max_colwidth', 400)

from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from datasets import load_metric

if torch.cuda.is_available(): 
    device = 'cuda'  
else: 
    device ='cpu'

MANUAL_SEED = 2711

def deterministic(rep=True, manual_seed=MANUAL_SEED):
    if rep:
        torch.manual_seed(manual_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(manual_seed)
            torch.cuda.manual_seed_all(manual_seed)
        torch.backends.cudnn.enabled = False 
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        print(f'Experimento deterministico, seed: {manual_seed}')
        if device == 'cuda':
            print(f'Existe {torch.cuda.device_count()} GPU\
            {torch.cuda.get_device_name(0)} disponível.')
    else:
        print('Experimento randomico')
deterministic()    

Experimento deterministico, seed: 2711
Existe 1 GPU            Tesla T4 disponível.


In [9]:
test_dataset = load_dataset('glue', 'sst2', split='validation')
test_dataset

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 872
})

In [28]:
shots = [
    {"text":"hide new secretions from the parental units ","label":"negative"},
    {"text":"contains no wit , only labored gags ", "label":"negative"},
    {"text":"that loves its characters and communicates something rather beautiful about human nature ","label":"positive"}
]

def generate_static_prompt(data_text, num_fewshot=3):

    if num_fewshot > 3:
        return "Fewshot error must be bellow than 3"
    
    elif num_fewshot==0:
        labeled_examples = ""
    
    else:
        fewshotex = shots[:num_fewshot]

        labeled_examples = "Given the examples below, answer after ##label: the sentence as positive or negative:\n"
        for i, doc_shot in enumerate(fewshotex):
            labeled_examples += f"Text {i+1}:" + " " + doc_shot['text'] + " " + f"##label: {doc_shot['label']}\n"
        labeled_examples += f'Text {len(fewshotex) + 1}: '
      
        example = data_text + "##label:"
    
    return labeled_examples + example

three_shot_inputs = []
for text, label in zip(test_dataset['sentence'], test_dataset['label']):
    three_shot_inputs.append(generate_static_prompt(text, num_fewshot=3))

print(three_shot_inputs[1])

Given the examples below, answer after ##label: the sentence as positive or negative:
Text 1: hide new secretions from the parental units  ##label: negative
Text 2: contains no wit , only labored gags  ##label: negative
Text 3: that loves its characters and communicates something rather beautiful about human nature  ##label: positive
Text 4: unflinchingly bleak and desperate ##label:


In [29]:
model_bloom = 'bigscience/bloomz-7b1-mt'
tokenizer_bloom = AutoTokenizer.from_pretrained(model_bloom)

# load bloom in 8bits with bitsandbytes
BLOOM_model = AutoModelForCausalLM.from_pretrained(model_bloom, device_map="auto", load_in_8bit=True)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

# Run

In [ ]:
deterministic() 

trues, preds = [], []
loop = tqdm(three_shot_inputs, leave=True)

for ix, batch in enumerate(loop):
    inputs = tokenizer_bloom.encode(three_shot_inputs[ix], return_tensors="pt", max_length=1024).to(device)
    outputs = BLOOM_model.generate(inputs)
    preds.append(tokenizer_bloom.decode(outputs[0]))
    trues.append(test_dataset['label'][ix])    

# Model Evaluation



In [37]:
#checking predictions careful to not cut the string predictions
dataframe = pd.DataFrame({'true': trues, 'pred':preds})
dataframe.pred.apply(lambda x: x[-8:])

0      positive
1      positive
2      positive
3      positive
4      negative
         ...   
867    negative
868    positive
869    negative
870    negative
871    positive
Name: pred, Length: 872, dtype: object

In [38]:
dataframe['pred'] = dataframe.pred.apply(lambda x: x[-8:])
dataframe.pred.apply(lambda x: 1 if x == "positive" else 0)

0      1
1      1
2      1
3      1
4      0
      ..
867    0
868    1
869    0
870    0
871    1
Name: pred, Length: 872, dtype: int64

In [39]:
 dataframe['pred'] = dataframe.pred.apply(lambda x: 1 if x == "positive" else 0)
 dataframe

,true,pred
0,1,1
1,0,1
2,1,1
3,1,1
4,0,0
...,...,...
867,0,0
868,1,1
869,0,0
870,0,0


In [41]:
metric = load_metric("accuracy")
metric.compute(predictions=dataframe['pred'], references=dataframe['true'])

<ipython-input-41-a85457dc0b5b>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric= load_metric("accuracy")


{'accuracy': 0.908256880733945}

Using **few-shot** examples:

# Try it yourself

You can try sentiment analysis with your own sentences by providing the input text as a string to the **`text`** variable in the code snippet provided below.


In [46]:
prompt = "Given the text classify the sentence as positive or negative:\n" 
text = prompt + "This movie is a bad movie!" #@param 
inputs = tokenizer_bloom.encode(text, return_tensors="pt", max_length=1024).to(device)
outputs = BLOOM_model.generate(inputs)
tokenizer_bloom.decode(outputs[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Given the text classify the sentence as positive or negative:\nThis movie is a bad movie! negative'